In [ ]:
from tinyec import registry
import secrets

curve = registry.get_curve('brainpoolP256r1')

def compress_point(point):
    return hex(point.x) + hex(point.y % 2)[2:]


#--------------------------------------------------------------------
#   Generate CLOUD_PRIVATE_KEY and CLOUD_PUBLIC_KEY

privKey = secrets.randbelow(curve.field.n)
pubKey = privKey * curve.g

#--------------------------------------------------------------------


def ecc_calc_decryption_key(privKey, ciphertextPubKey):
    sharedECCKey = ciphertextPubKey * privKey
    return sharedECCKey

#--------------------------------------------------------------------
#   Connect to an MQTT brocker and publish a topic
    #PUBLISH TOPIC
    #SEND THE CLOUD_PUBLIC_KEY

#--------------------------------------------------------------------
#   Connect to an MQTT brocker and subscribe to a topic
    #SUBSCRIBE TOPIC
    #RECEIVE THE CLIENT_PUBLIC_KEY AND THE ENCRYPTED_DATA
    #GENERATE SHARED_KEY BY COUPLE (CLOUD_PRIVATE_KEY,CLIENT_PUBLIC_KEY)



#--------------------------------------------------------------------
#   Here you can generate your file


#   Dencrypt the file using ECC cryption


#   Send the file through MQTT to publish data



#   END OF CODE

